# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [98]:
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [99]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()

100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

$ \begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align} $

However, we don't know $ Q^* $. So, we use neural network as a approximators, we can simply create one and train it to resemble $ Q^* $.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

$ \begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align} $

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

$ \begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align} $

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [100]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size, hidden_size=100):
        super(Model, self).__init__()
        # initialise layers here
        self.dense_layer1 = nn.Linear(observation_size, hidden_size)
        self.dense_layer2 = nn.Linear(hidden_size, action_size)

    def forward(self, x):
        # send x through the network
        x = self.dense_layer1(x)
        x = self.dense_layer2(x)
        return x

    # This is to have a function that outputs an action with the highest q-value
    def predict(self, x):
        x = self.forward(x)
        return x


### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [101]:
import queue
import random

from torch import tensor

class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.memory = [] # memory that stores N most new transitions
        # good place to store hyperparameters as attributes
        self.learning_rate = 0.7           # Learning rate
        self.gamma = 0.9                   # Discounting rate - Old value: 0.618

        self.total_episodes = 5000        # Total episodes
        # total_test_episodes = 5       # Total test episodes
        # max_steps = 99                # Max steps per episode

        # # Exploration parameters
        self.epsilon = 1.0                 # Exploration rate
        # max_epsilon = 1.0             # Exploration probability at start
        # min_epsilon = 0.01            # Minimum exploration probability
        # decay_rate = 0.0001             # Exponential decay rate for exploration prob

    def remember(self, state, action, reward, next_state, done):
        # add to memory

        print(state)
        print(action)
        print(reward)
        print(next_state)
        
        transition = (state, action, next_state, reward, done)
        if len(self.memory) > 10000:
            del self.memory[0]
            self.memory.append(transition)
        else:
            self.memory.append(transition)


    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def act(self, state):
        # return an action from the model
        # Doing a random choice --> exploration
        if random.uniform(0, 1) < self.epsilon:
            action = env.action_space.sample()
        # Else --> exploitation (taking the biggest Q value for this state)
        else:
            action = torch.argmax(self.model.predict(torch.tensor(state)))
            action = int(action)
        return action


    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        transitions = self.sample(batch_size)
        for state, action, reward, next_state, done in transitions:

            q_current = self.model.forward(torch.tensor(state, dtype=torch.float32))[action]
            #self.model.forward(torch.tensor(state))
            #float(torch.max(self.model.forward(torch.tensor(state))))
            #self.model.forward(torch.tensor(state))[action]
            q_target = reward + (torch.max(self.model.forward(torch.tensor(next_state, dtype=torch.float32))) * int(not done))
            # float(torch.max(self.model.forward(torch.tensor(state))))
            
            #value = float(torch.max(self.model.forward(torch.tensor(state))))

            loss = self.criterion(q_current, q_target)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


In [102]:
int(not False)

1

### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [103]:
def train(env, agent, episodes=1000, batch_size=64):  # train for many games
    for _ in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        while not done:
            # 1. make a move in game.
            action = agent.act(state)
            next_state, reward, done, truncated, info = env.step(action)
            # 2. have the agent remember stuff.
            agent.remember(state, action, reward, next_state, done)
            # 3. update state
            state = next_state
            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
    env.close()

### Putting it together
---
We train an agent on the environment:


In [104]:
env = gym.make('CartPole-v1')  # , render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
# torch.save(agent.model.state_dict(), 'model.pth')

  0%|          | 2/1000 [00:00<00:07, 125.34it/s]

[-0.01417059 -0.00935043  0.01676764 -0.03222782]
0
1.0
[-0.0143576  -0.20470877  0.01612308  0.26569796]
[-0.0143576  -0.20470877  0.01612308  0.26569796]
0
1.0
[-0.01845177 -0.40005708  0.02143704  0.5634223 ]
[-0.01845177 -0.40005708  0.02143704  0.5634223 ]
0
1.0
[-0.02645291 -0.5954732   0.03270549  0.8627811 ]
[-0.02645291 -0.5954732   0.03270549  0.8627811 ]
1
1.0
[-0.03836238 -0.4008114   0.04996111  0.5805583 ]
[-0.03836238 -0.4008114   0.04996111  0.5805583 ]
0
1.0
[-0.04637861 -0.59659654  0.06157227  0.888552  ]
[-0.04637861 -0.59659654  0.06157227  0.888552  ]
1
1.0
[-0.05831053 -0.4023618   0.07934331  0.6158428 ]
[-0.05831053 -0.4023618   0.07934331  0.6158428 ]
1
1.0
[-0.06635777 -0.20843281  0.09166017  0.34916696]
[-0.06635777 -0.20843281  0.09166017  0.34916696]
0
1.0
[-0.07052643 -0.4047307   0.09864351  0.66928816]
[-0.07052643 -0.4047307   0.09864351  0.66928816]
0
1.0
[-0.07862104 -0.6010758   0.11202927  0.99132824]
[-0.07862104 -0.6010758   0.11202927  0.991328

RuntimeError: both arguments to matmul need to be at least 1D, but they are 0D and 2D

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/